# Dynamic Vertical Triplet Energy Generation (DvTEGen)

## Import Dependencies

In [2]:
from pathlib import Path
from aqme.qprep import qprep
import os
import shutil

ModuleNotFoundError: No module named 'aqme'

## Define Parameters for QM input file generation

In [3]:
#Step Number for Coordinate Extraction

step_size = 8 #Step size for trajectory sampling (Default = 8)

i_step = 1 #First Step in the sampled trajectories (Default = 1)

f_step = 1000 #Final Step in the sampled trajectories (Default = 1000)

#QM inputs

qm_input='m062x/6-31G* scf=xqc' #Level of theory for vertical Triplet Enegy calculations

program='gaussian' #Program used for QM calculations (Options: gaussian and orca)

mem='64GB' #Memory used (See ACME specifications)

nprocs=32 #Number of processors (See ACME specifications)

# Generation of QM input files

Note: The code assume that .xyz trajectory files are in the working directory.
      Refer to the code comments how to specify other workign folders

In [4]:
print('Selected Parameters:')
print()
print('step_size = ' + str(step_size))
print('i_step = ' + str(i_step))
print('f_step = ' + str(f_step))

print('Level of Theory = ' + str(qm_input))
print('Program = ' + str(program))
print('Memory = ' + str(mem))
print('Number of Processors = ' + str(nprocs))
print()

#Load .XYZ trajectories in current folder
xyz_files = [str(filepath) for filepath in Path().glob('*.xyz')]   #Specify within Path() the desired working directory
                                                                   #Change extension to .trj if using JProgDyn outputs

#Remove files that may have already been parsed
xyz_files = [item for item in xyz_files if not item.startswith("parsed")]

print('Loaded Filed:')
print()
[print(file) for file in xyz_files]

#Extract Number of Atoms in trajectories
try:
    # Open the file in read mode
    with open(xyz_files[0], 'r') as file:
        # Read the first line
        atom_no = int(file.readline())

        # Print the first line
        print()
        print("Total number of atoms in trajectory:", atom_no)
        print()
        print("Note: It is assumed that the trajectories have the same number of atoms!")

except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print("An error occurred:", str(e))
    
#Extract Coordinates for a given step number:
def extract_coordinates(file_name, file_out):
    try:
        with open(file_name, 'r') as file:
            with open(file_out, 'w') as fileout:
                for x in range (1, 1000, step_size):
                    search_string = 'Step ' + str(x)
                    found = False
                    for line in file:
                        if search_string in line:
                            found = True
                            fileout.write(str(atom_no))
                            fileout.write(str('\n'))
                            fileout.write(line.rstrip())
                            fileout.write(str('\n'))
                            break

                    if found:
                        lines_to_print = atom_no
                        for _ in range(lines_to_print):
                            line = next(file, None)
                            if line:
                                fileout.write(line.rstrip())
                                fileout.write(str('\n'))
                            else:
                                break
                    else:
                        print("String not found.")
    except FileNotFoundError:
        print("File not found.")
        
#Apply Coordinate extractors to files in folder
print()
print("Parsed Trajectories Generated:")
print()
xyz_parsed = []
for i in xyz_files:
    file_out = 'parsed-' + i
    extract_coordinates(i, file_out)
    print(file_out)
    xyz_parsed.append(file_out)
    
#Execution of Input Files Generator

print()
print('Initiating AQME QPREP for S0 States:')
print()

qprep(files=xyz_parsed, qm_input=qm_input, suffix='S0', mult=1, program=program, mem=mem, nprocs=nprocs)

print()
print('Initiating AQME QPREP for T1 States:')
print()

qprep(files=xyz_parsed, qm_input=qm_input, suffix='T1', mult=3, program=program, mem=mem, nprocs=nprocs)

#Due to how AQME names files, they will be renamed to resemble the entry in the original trajectory file

directory = 'QCALC'  # Replace with the directory path containing the files

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.com'):
        parts = filename.split('_')

        # Extract the x and y values
        molecule = parts[0]
        trajectory = parts[1]
        frame = int(parts[3])
        state = parts[4]

        # Calculate the new value of y
        new_frame = ((frame - i_step) * step_size) + i_step

        # Create the new filename
        new_filename = f"{molecule}_{trajectory}_frame_{new_frame}_{state}"

        # Rename the file
        old_path = os.path.join(directory, filename)
        new_path = os.path.join(directory, new_filename)
        os.rename(old_path, new_path)

        #print(f"Frame file generated: {new_filename}")
        
print('Files have been generated Succesfully!')
print()
print('There are a total of ' + str(len(os.listdir(directory))) + ' Files. Good Luck!')

Selected Parameters:

step_size = 8
i_step = 1
f_step = 1000
Level of Theory = m062x/6-31G* scf=xqc
Program = gaussian
Memory = 64GB
Number of Processors = 32

Loaded Filed:

ethene-midi-200_90230434612.xyz
ethene-midi-200_32710575084.xyz
ethene-midi-200_81009192108.xyz
ethene-midi-200_17181703104.xyz
ethene-midi-200_55064996766.xyz
ethene-midi-200_45325129809.xyz
ethene-midi-200_55911034846.xyz
ethene-midi-200_36065760166.xyz
ethene-midi-200_46201230678.xyz
ethene-midi-200_60662191247.xyz
ethene-midi-200_24878406129.xyz
ethene-midi-200_88761288262.xyz
ethene-midi-200_49776493491.xyz
ethene-midi-200_47045274548.xyz
ethene-midi-200_21293896057.xyz
ethene-midi-200_83370062741.xyz
ethene-midi-200_89405535063.xyz
ethene-midi-200_70835075668.xyz
ethene-midi-200_24664528333.xyz
ethene-midi-200_84856749744.xyz
ethene-midi-200_50449081487.xyz
ethene-midi-200_12951521826.xyz
ethene-midi-200_54331752561.xyz
ethene-midi-200_56626846535.xyz
ethene-midi-200_41436213944.xyz

Total number of atoms in

NameError: name 'qprep' is not defined

## (optional) batch files based on initial dynamics trajectory

When using HPC resources it may be easier to batch files for submission for efficient usage. If trajecotries are generated using MILO, this cell will compile them in dividiual folder named after the trajectory random seed

In [ ]:
# Define the source directory where the files are located
source_directory = 'QCALC'

# Define the destination directory where the files will be moved
destination_directory = 'QCALC/Batched/'

# Iterate over the files in the source directory
for filename in os.listdir(source_directory):
    y = filename.split('_')[-4]  # Assuming the y variable is at index 3
        
    # Create the destination folder based on the y value
    destination_folder = os.path.join(destination_directory, y)
    os.makedirs(destination_folder, exist_ok=True)
        
    # Move the file to the destination folder
    source_file = os.path.join(source_directory, filename)
    destination_file = os.path.join(destination_folder, filename)
    shutil.move(source_file, destination_file)
    
Print('Batching based on trajecotory completed')